# CPSC 330 - Applied Machine Learning 

## Homework 5: Evaluation metrics
### Associated lectures: [Lectures 9, 10](https://ubc-cs.github.io/cpsc330/README.html) 

**Due date: Monday, Feb 27, 2023 at 11:59pm**

## Imports

In [2]:
import os
import re
import sys
from hashlib import sha1

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import tests_hw5
from sklearn import datasets
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

## Instructions 
<hr>
rubric={points:3}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2022W2/blob/main/docs/homework_instructions.md). 

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).

## Exercise 1: Precision, recall, and f1 score by hand <a name="1"></a>
<hr>

Consider the problem of predicting whether a patient has cancer or not. It is important to catch this disease early to reduce mortality rate; late diagnosis will result in metastasis to other organs, which adversely impacts patient's prognosis. Below are confusion matrices of two machine learning models: Model A and Model B. 

- Model A

|         | Predicted disease | Predicted no disease |
| :------------- | -----------------------: | -----------------------: |
| **Actual disease**       | 48 | 32 |
| **Actual no disease**       | 20 | 100 |


- Model B

|        | Predicted disease | Predicted no disease |
| :------------- | -----------------------: | -----------------------: |
| **Actual disease**       | 43 | 22 |
| **Actual no disease**       | 35 | 100 |

### 1.1 Positive vs. negative class 
rubric={points:2}

**Your tasks:**

Precision, recall, and f1 score depend upon which class is considered "positive", that is the thing you wish to find. In the example above, which class is likely to be the "positive" class? Why? 

Disease is likely the positive class, because that is the thing we are searching for. Moreover, TN usually is the one with the highest value, and so no disease would be the negative class.  

### 1.2 Accuracy
rubric={points:2}

**Your tasks:**

Calculate accuracies for Model A and Model B. 

We'll store all metrics associated with Model A and Model B in the `results_dict` below. 

In [3]:
results_dict = {"A": {}, "B": {}}

In [4]:
results_dict["A"]["accuracy"] = 0.74
results_dict["B"]["accuracy"] = 0.715

### 1.3 Which model would you pick? 
rubric={points:1}

**Your tasks:**

Which model would you pick simply based on the accuracy metric? 

Simply based on the accuracy metric, I would use model A, as it has a higher accuracy value. 

### 1.4 Precision, recall, f1-score
rubric={points:6}

**Your tasks:**

1. Calculate precision, recall, f1-score for Model A and Model B manually, without using `scikit-learn` tools. 


In [5]:
results_dict["A"]["precision"] = 48/68
results_dict["B"]["precision"] = 43/78
results_dict["A"]["recall"] = 48/80
results_dict["B"]["recall"] = 43/65
results_dict["A"]["f1"] = 24/37
results_dict["B"]["f1"] = 86/143

Show the dataframe with all results. 

In [6]:
pd.DataFrame(results_dict)

,A,B
accuracy,0.740000,0.715000
precision,0.705882,0.551282
recall,0.600000,0.661538
f1,0.648649,0.601399


### 1.5 Discussion
rubric={points:4}

**Your tasks:**
1. Given the type of problem (early cancer diagnosis), which metric is more informative in this problem? Why? 
2. Which model would you pick based on this information? 

ANSWERS: 
1. Given the fact that the type of problem is early cancer diagnosis, I would say recall is more informative. This is because false negatives are more damaging, since the worst case scenario is when we tell a patient of no disease, but there actually is a disease. 

2. Based on the above info, I would want a model with the higher recall score, therefore I would pick model B. 

### (Optional) 1.6 
rubric={points:1}

**Your tasks:**

Provide 2 to 3 example classification datasets (with links) where accuracy metric would be misleading. Discuss which evaluation metric would be more appropriate for each dataset. You may consider datasets we have used in this course so far. You could also look up datasets on Kaggle. 

### Exercise 2: Classification evaluation metrics using `sklearn` <a name="2"></a>
<hr>

In general, when a dataset is imbalanced, accuracy does not provide the whole story. In class, we looked at credit card fraud dataset which is a classic example of an imbalanced dataset. 

Another example is customer churn datasets. [Customer churn](https://en.wikipedia.org/wiki/Customer_attrition) refers to the notion of customers leaving a subscription service like Netflix. In this exercise, we will try to predict customer churn in a dataset where most of the customers stay with the service and a small minority cancel their subscription. To start, please download the [Kaggle telecom customer churn dataset](https://www.kaggle.com/becksddf/churn-in-telecoms-dataset). Once you have the data, you should be able to run the following code:

The starter code below reads the data CSV as a pandas dataframe and splits it into 70% train and 30% test. 

Note that `churn` column in the dataset is the target. "True" means the customer left the subscription (churned) and "False" means they stayed (unchurned).

> Note that for this kind of problem a more appropriate technique is something called survival analysis and we'll be talking about it later in the course. For now, we'll just treat it as a binary classification problem. 

In [7]:
df = pd.read_csv("bigml_59c28831336c6604c800002a.csv", encoding="utf-8")
train_df, test_df = train_test_split(df, test_size=0.3, random_state=123)
train_df
df

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False


### 2.1 Distribution of target values
rubric={points:4}

**Your tasks:**

Examine the distribution of target values in the train split. Do you see class imbalance? If yes, do we need to deal with it? Why or why not? 

In [8]:
train_df["churn"].value_counts(normalize=True)

False    0.850407
True     0.149593
Name: churn, dtype: float64

We can see that 15% of the data contains churned (left the subscription), while 85% of the data is not churned (they kept the subscription).
I do see class imbalance, a 85-15 split is quite a big difference. There are many more examples of unchurned compared to churned.
We need to deal with this class imbalance, otherwise our model might bias towards the majority class (unchurned), which might lead to unreliable classification of the minority class (churned).\

### (Optional) 2.2 EDA 
rubric={points:1}

**Your tasks:**

Come up with **two** exploratory questions you would like to answer and explore those. Briefly discuss your results in 1-3 sentences.

You are welcome to use `pandas_profiling` (see Lecture 10) but you don't have to.

### 2.3 Column transformer 
rubric={points:14}

The code below creates `X_train`, `y_train`, `X_test`, `y_test` for you. 
In preparation for building a classifier, set up a `ColumnTransformer` that performs whatever feature transformations you deem sensible. This can include dropping features if you think they are not helpful. Remember that by default `ColumnTransformer` will drop any columns that aren't accounted for when it's created.

For each group of features (e.g. numeric, categorical or else) explain why you are applying the particular transformation. For example, "I am doing transformation X to the following categorical features: `a`, `b`, `c` because of reason Y," etc.

Finally, fit `ColumnTransformer` on your training set; and use the `ColumnTransformer` to transform your train data.

In [9]:
X_train = train_df.drop(columns=["churn"])
X_test = test_df.drop(columns=["churn"])
y_train = train_df["churn"]
y_test = test_df["churn"]

numeric_features = ['account length', 'number vmail messages', 'total day minutes' , 'total day charge', 'total eve minutes' , 'total eve charge' , 'total night minutes' , 'total night charge', 'total intl minutes' , 'total intl charge', 'customer service calls']
categorical_features = ['state']
ordinal_features = []
binary_features = ['international plan' , 'voice mail plan']
drop_features = ['area code', 'phone number' , 'total day calls' , 'total eve calls' , 'total night calls' , 'total intl calls']
passthrough_features = []
target = "churn"


ct = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown="ignore", sparse=False), categorical_features),
    (OneHotEncoder(drop="if_binary", dtype=int), binary_features),  
    ("drop", drop_features),
)


X_train_pp = ct.fit_transform(X_train)

# ct

column_names = (
    numeric_features
    + ct.named_transformers_["onehotencoder-1"].get_feature_names_out().tolist()
    + ct.named_transformers_["onehotencoder-2"].get_feature_names_out().tolist()
)

X_train_pp = pd.DataFrame(X_train_pp, columns=column_names)
X_train_pp

# train_df.describe()
# train_df.info()
# df['state'].unique()
# len(numeric_features)

,account length,number vmail messages,total day minutes,total day charge,total eve minutes,total eve charge,total night minutes,total night charge,total intl minutes,total intl charge,...,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,international plan_yes,voice mail plan_yes
0,-0.767893,-0.587624,0.618769,0.618927,0.069871,0.069926,1.088667,1.089926,-1.645501,-1.644994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.843585,-0.587624,-1.293778,-1.293517,-1.167277,-1.166291,-2.162302,-2.164029,0.227019,0.222249,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.544113,1.900976,-0.609809,-0.609654,-2.210130,-2.211244,0.369287,0.369252,1.919489,1.916105,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.165650,-0.587624,0.998345,0.998611,-0.754894,-0.755774,1.597736,1.596582,-0.097071,-0.097850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.115188,-0.587624,-0.994886,-0.994731,1.195994,1.196515,0.793839,0.792921,0.407069,0.408973,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,0.645037,-0.587624,0.329045,0.328771,0.898602,0.897957,1.652771,1.653363,0.803179,0.809096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2329,-0.767893,1.608200,-0.664820,-0.664665,-0.618094,-0.618157,-1.440956,-1.438987,-0.313131,-0.311249,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2330,0.619806,-0.587624,0.048489,0.048322,-1.216842,-1.217606,-0.582024,-0.582913,0.479089,0.475660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2331,1.477656,-0.587624,0.173180,0.173445,0.890672,0.890960,0.801701,0.801656,0.046969,0.048862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


drop_features:
    the phone number a user has doesn't really affect whether they will leave or not. For example, what difference will it make if the user had 403-681-5975 vs 403-579-5186?
    day, eve, night, and intl calls: While the number of calls may be important, the dataset already has the number of minutes for each of those categories, so the number of calls can be omitted. 
    area code isn't really helpful if I already know which state they're in, it's arguably more precise and gives more insight on a closer level.
    

### 2.4 area code feature
rubric={points:4}

The original dataset had a feature called `area code`.

1. The area codes are numbers. Does it make sense to encode them as one-hot-endoded (OHE) or not? Please justify your response.
2. What were the possible values of `area code`? 
3. If area code is encoded with OHE, how many new features are created to replace it?

1. Yes, it would make sense to encode them as OHE. Even though they are numbers, what they really represent are categories (there's 3 unique values in 'area code'. Since they're categories, we should use OHE to represent them.)
2. The possible values of area code are 415, 408, and 510. 
3. If OHE was applied to area code, there will be 2 more additional columns, but there's technically 3 new features. (all are renamed to something different, for example, 'area code' may be renamed to 'area code_408')

In [10]:
df['area code'].unique()

array([415, 408, 510], dtype=int64)

### 2.5 Logistic regression
rubric={points:12} 

**Your tasks:**

1. Report the cross-validation results of a `LogisticRegression` model, with default Hparams, on the following metrics: `"accuracy", "precision", "recall", "f1"`
2. Are you satisfied with the results? Explain why or why not. Discuss in a few sentences. 

In [22]:
scoring = ["accuracy", "precision", "recall", "f1"]

pipe = make_pipeline(StandardScaler(), LogisticRegression())
scores = cross_validate(pipe, X_train_pp, y_train, return_train_score=True, scoring=scoring)
pd.DataFrame(scores).mean()

fit_time           0.012979
score_time         0.003461
test_accuracy      0.848260
train_accuracy     0.862302
test_precision     0.485133
train_precision    0.596521
test_recall        0.203685
train_recall       0.248574
test_f1            0.284321
train_f1           0.350423
dtype: float64

The average recall scores seem pretty low, I'm not satisfied with the results. 

### 2.6 Logistic regression with `class_weight`
rubric={points:6}

**Your tasks:**

1. Set the `class_weight` parameter of your logistic regression model to `'balanced'` and report the same metrics as in the previous part. 
2. Do you prefer this model to the one in the previous part? Discuss your results in a few sentences while comparing the metrics of this model and the previous model.

In [23]:
scoring = ["accuracy", "precision", "recall", "f1"]

pipe = make_pipeline(StandardScaler(), LogisticRegression(class_weight="balanced"))
scores = cross_validate(pipe, X_train_pp, y_train, return_train_score=True, scoring=scoring)
pd.DataFrame(scores).mean()

fit_time           0.010957
score_time         0.004264
test_accuracy      0.755661
train_accuracy     0.776467
test_precision     0.344612
train_precision    0.381580
test_recall        0.693458
train_recall       0.795835
test_f1            0.459812
train_f1           0.515823
dtype: float64

I prefer this model to the previous one, because the quality of all metrics overall have improved. Some have decreased slightly, but the ones that increased, increased significantly. 

### 2.7 Hyperparameter optimization
rubric={points:10}

1. Jointly optimize `C` and `class_weight` with `GridSearchCV` and `scoring="f1"`.
  - For `class_weight`, consider 3 values: 
    - `None` (no weight)
    - "weight of class 0 = 1"  and  "weight of class 1 = 3"
    - '`balanced`'
  - For `C`, choose some reasonable values
2. What values of `C` and `class_weight` are chosen and what is the best cross-validation f1 score?

In [24]:
pipe = make_pipeline(StandardScaler(), LogisticRegression(**best_parameters))

param_grid = {
        ""
}

pipe

NameError: name 'best_parameters' is not defined


<br><br>

### 2.8 Test results
rubric={points:10}

**Your tasks**
1. Evaluate the best model on the test set. In particular show each of the following on the test set:  
    - Plot Confusion matrix
    - Plot Precision-recall curve 
    - Calculate average precision score
    - Plot ROC curve
    - Report AUC score
3. Comment on the AUC score and give an intuitive explanation of what this value of AUC means for this problem.

<br><br><br><br>

### Exercise 3: Regression metrics <a name="3"></a>
<hr> 


For this exercise, we'll use [California housing dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) from `sklearn datasets`. The code below loads the dataset.  

In [3]:
from sklearn.datasets import fetch_california_housing

housing_df = fetch_california_housing(as_frame=True).frame
housing_df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


### 3.1: Data spitting and exploration 
rubric={points:4}

**Your tasks:**

1. Split the data into train (75%) and test (25%) splits. 
2. Explore the train split. Do you need to apply any transformations on the data? If yes, create a preprocessor with the appropriate transformations. 
3. Separate `X` and `y` to train and test splits. 

In [26]:
train_df, test_df = train_test_split(housing_df, test_size=0.25, random_state=35)

print(train_df)

X_train, y_train = train_df.drop(columns=['MedHouseVal']) , train_df['MedHouseVal']
X_test, y_test = test_df.drop(columns=['MedHouseVal']) , test_df['MedHouseVal']

       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
14178  2.5682      36.0  5.440000   1.055556      1268.0  2.817778     32.71   
9256   1.4267      37.0  4.291845   1.120172      1092.0  4.686695     36.96   
371    3.1458      41.0  4.481675   0.931937       571.0  2.989529     37.74   
5168   1.5000      35.0  4.230612   1.008163      1753.0  3.577551     33.95   
15029  2.9896      20.0  4.518135   1.031088      1532.0  2.645941     32.78   
...       ...       ...       ...        ...         ...       ...       ...   
19391  2.2679      25.0  3.971698   0.962264       303.0  2.858491     37.78   
15393  3.1906      11.0  5.877667   1.099573      2012.0  2.862020     33.22   
9143   4.5769      32.0  6.341137   1.143813       806.0  2.695652     34.48   
17679  4.3419      22.0  5.688679   1.096226      2491.0  4.700000     37.32   
5833   3.9866      36.0  6.360129   1.083601       834.0  2.681672     34.20   

       Longitude  MedHouseVal  
14178  

### 3.2 Baseline: Linear Regression 
rubric={points:2}

**Your tasks:**
1. Carry out cross-validation using `sklearn.linear_model.LinearRegression` with default scoring. 
2. What metric is used for scoring by default? 

In [27]:
ln = LinearRegression();

lr_pipe = make_pipeline(StandardScaler(), ln)
pd.DataFrame(cross_validate(lr_pipe, X_train, y_train, cv=5, return_train_score=True)).mean()

# lr_pipe = make_pipeline(ct, LinearRegression())
# pd.DataFrame(cross_validate(lr_pipe, X_train, y_train, cv=10, return_train_score=True))

fit_time       0.016408
score_time     0.001591
test_score     0.603310
train_score    0.607485
dtype: float64

$R^2$ is the metric used for scoring, by default. 

### 3.3 Random Forest Regressor
rubric={points:7}

In this exercise, we are going to use [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) model which we haven't looked into yet. At this point you should feel comfortable using models with our usual ML workflow even if you don't know the details. We'll talk about `RandomForestRegressor` later in the course.  

The code below defines a custom scorer called `mape_scorer` and creates dictionaries for two model (`models`) and five evaluation metrics (`score_types_reg`). 

**Your tasks:**

1. Using the `models` and the evaluation metrics `score_types_reg` in the code below, carry out cross-validation with each model, by passing the evaluation metrics to `scoring` argument of `cross_validate`. Use a pipeline with the model as an estimator if you are applying any transformations. 
2. Show results as a dataframe. 
3. Interpret the results. How do the models compare to the baseline? Which model seems to be performing well with different metrics? 


In [28]:
models = {
    "Ridge": Ridge(),
    "Random Forest": RandomForestRegressor(),
}

score_types_reg = {
    "neg_mean_squared_error": "neg_mean_squared_error",
    "neg_root_mean_squared_error": "neg_root_mean_squared_error",
    "neg_mean_absolute_error": "neg_mean_absolute_error",
    "r2": "r2",
    "neg_mean_absolute_percentage_error": "neg_mean_absolute_percentage_error",
}

model_scores = {"Ridge": {}, "Random Forest": {}}


for model in models:
    pipe = make_pipeline(
        StandardScaler(),
        models[model])
    for score in score_types_reg:
        scores = cross_validate(pipe, X_train, y_train, cv=5, scoring=score_types_reg[score], return_train_score=True, n_jobs=-1)
        model_scores[model][score] = scores["test_score"].mean()

model_scores = pd.DataFrame(model_scores)
model_scores

,Ridge,Random Forest
neg_mean_squared_error,-0.528073,-0.262673
neg_root_mean_squared_error,-0.726149,-0.511737
neg_mean_absolute_error,-0.531165,-0.333973
r2,0.603314,0.802133
neg_mean_absolute_percentage_error,-0.318518,-0.189949


<br><br>

### 3.4 Hyperparameter optimization 
rubric={points:1}

1. Carry out hyperparameter optimization using `RandomizedSearchCV` and `Ridge` with the following `param_dist`. The `alpha` hyperparameter of `Ridge` controls the fundamental tradeoff. Choose `neg_mean_absolute_percentage_error` as the HParam optimization metric.

2. What was the best `alpha` hyper-parameter found?

In [ ]:
from scipy.stats import loguniform

param_dist = {"ridge__alpha": loguniform(1e-3, 1e3)}

<br><br>

### 3.5 Test results
rubric={points:4}

**Your tasks:**

Test the best model (from 3.4) on the test set based on the `neg_mean_absolute_percentage_error` score.

<br><br>

### 3.6 Model interpretation  
rubric={points:4}

Ridge is a linear model and it learns coefficients associated with each feature during `fit()`. 

**Your tasks:**

1. Explore coefficients learned by the `Ridge` model above as a pandas dataframe with two columns: 
   - features 
   - coefficients
2. Increasing which feature values would result in higher housing price? 

<br><br>

## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 